In [1]:
import math

#done
def convert_to_float(not_float):
    float_list = []
    for i in not_float:
            float_list.append([float(j) for j in i])
    return float_list

In [2]:
def process_file(raw_file_name):
    raw_file = open(raw_file_name, 'r')
    str_data = []
    with open(raw_file_name) as file:
        raw_data = file.readlines()
    str_data = [i.split() for i in raw_data]
    return convert_to_float(str_data)

In [3]:
#done
def take_out_classification(data):
    classfication = []
    for i in data:
        classfication.append(i.pop(0))
    return classfication

In [4]:
#done
def add_back_classification(data, list_of_classfication):
    for i in range(0, len(data)):
        data[i].insert(0, float(list_of_classfication[i]))

In [5]:
#done
def getmean(data, features, instances):
    mean_list = []
    column = 0
    #classif = take_out_classification(data)
    for i in range(features):
        mean = sum(row[column] for row in data)/instances
        column = column + 1
        mean_list.append(mean)
    #add_back_classification(data, classif)
    return mean_list

In [6]:
#done
def getstd(data, mean_list, features, instances):
    standard_d_list = []
    #classif = take_out_classification(data)
    mean_list = getmean(data, features, instances)
    for i in range(features):
        standard_d = math.sqrt((sum(pow((row[i] - mean_list[i]), 2) for row in data)) / instances)
        standard_d_list.append(standard_d)
    #add_back_classification(data, classif)
    #df = pd.DataFrame(standard_d_list)
    #print(df)
    return standard_d_list

In [7]:
#done
def scale(data):
    classif = take_out_classification(data)
    features = len(data[0])
    instances = len(data)
    #print(instances)
    mean_list = getmean(data, features , instances)
    standard_d_list = getstd(data, mean_list, features, instances )
    #df  = pd.DataFrame(data)
    #print(df)
    
    for i in range(0, instances):
        for j in range(features):
            #print("in loop")
            #print(data[i][j])
            data[i][j] = (data[i][j] - mean_list[j])/ standard_d_list[j]
    add_back_classification(data, classif)
    return data

In [8]:
#done
def euclideanDistance(instance_a, instance_b, features):
    #remove classfication for calculation
    #print(instance_a)
    classification_1 = instance_a.pop(0)
    classification_2 = instance_b.pop(0)
    
    distance = 0
    #print(repr(instance_a) +"\n"+ repr(instance_b))
    for i in range(len(features)):
        #print(instance_b)
        #print(features[i])
        #print(instance_b[features[i]])
       # print("instance a = " +  repr(instance_a[i]) + " and instance b = " +  repr(instance_b[i]))
        distance = distance + pow((instance_a[features[i]] - instance_b[features[i]]), 2)
       # print("and the distance is: " + repr(distance) + "\n")
    
    #add classification back
    instance_a.insert(0, classification_1)
    instance_b.insert(0, classification_2)
    return math.sqrt(distance)

In [9]:
def nearest_neighbor(data, test, features):
    best_neighbor = [-1, -1]
    distance = float('inf')
    test_distance = float('inf')
    for i in data:
        if len(features) > 0:
            test_distance = euclideanDistance(i, test, features)
        if test_distance < distance:
            distance = test_distance
            #print(i[0])
            best_neighbor[0] = i[0]
            #print(test_distance)
            best_neighbor[1]= test_distance
    return best_neighbor

In [10]:
import numpy as np
def compute_accuracy(data,feature_set, feature):

    #print(feature)
    list_feature = list(feature_set)
    accuracy = 0
    test = []
    neighbor = []
    if feature < 0:
        feature_set.discard(abs(feature))
        list_feature = list(feature_set)
        feature_set.add(abs(feature))
    elif feature > 0:
        feature_set.add(abs(feature))
        list_feature = list(feature_set)
        feature_set.discard(abs(feature))
    #random(data)
    #count = 1
    for i in range(220):
        #count = count + 1
        np.random.shuffle(data)
        #print(data)
        test = data.pop(0)
        #print("test " + repr(test[0]))
        neighbor = nearest_neighbor(data, test, list_feature)
        #print("Neighbor " + repr(neighbor))
        #print("found " + repr(nearest_neighbor(data, test, feature)[0]))
        if  neighbor[0] == test[0]:
            accuracy = accuracy + 1
        data.append(test)
    #print(count)
    accuracy = accuracy/221
    return accuracy * 100

In [11]:
def forward_selection(data):
    print("Froward selection")
    features = len(data[0]) - 1
    feature_set = set()
    best_accuracy = 0
    #print(best_accuracy)
    for i in range(features):
        print("At level: " + repr(i+1))
        add_this = -1
        for j in range(i, features):
            if j not in feature_set:
                accuracy = compute_accuracy(data, feature_set, j)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    add_this = j
        if add_this > 0:
            feature_set.add(add_this)
            print("Best accuracy so far: " + "%.2f" % best_accuracy + " %" + " With feature set: " + repr(feature_set))
        else:
            print("(Warning, Accuracy has decreased! Continuing search in case of local maxima)")
    print("This is what I got: ")
    print("With feature set: " + repr(feature_set)+ " Best accuracy: " + "%.2f" % best_accuracy + " %")
            
    return feature_set

In [12]:
def backward_elimination(data):
    print("Backward elimination")
    features = len(data[0]) - 1
    feature_set = set(i for i in range(features))
    #print(feature_set)
    best_accuracy = 0
    #print(best_accuracy)
    for i in range(features):
        print("At level: " + repr(i+1))
        remove_this = -1
        for j in range(features):
            if j in feature_set:
                accuracy = compute_accuracy(data, feature_set, -1*j)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    remove_this = j
        if remove_this > 0:
            print("Removing feature: " + repr(remove_this))
            feature_set.discard(remove_this)
            print("Best accuracy so far: " + "%.2f" % best_accuracy + " %" + " With feature set: " + repr(feature_set))
        else:
            print("Warning, Accuracy has decreased! Breaking out!")
            break
    print("This is what I got: ")
    print("With feature set: " + repr(feature_set)+ " Best accuracy: " + "%.2f" % best_accuracy + " %")
    return feature_set

In [13]:
def custom_selection(data):
    feature_set = forward_selection(data)
    feature_set2 = set()
    feature_set2 = feature_set2.union(feature_set)
    
    features = len(data[0]) - 1
    #print(feature_set)
    best_accuracy = 0
    #print(best_accuracy)
    for i in range(features):
        print("At level: " + repr(i+1))
        remove_this = -1
        for j in range(features):
            if j in feature_set:
                accuracy = compute_accuracy(data, feature_set, -1*j)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    remove_this = j
        if remove_this > 0:
            print("Removing feature: " + repr(remove_this))
            feature_set.discard(remove_this)
            print("Best accuracy so far: " + "%.2f" % best_accuracy + " %" + " With feature set: " + repr(feature_set))
        else:
            print("Warning, Accuracy has decreased! Breaking out!")
            break
    print("This is what I got: ")
    print("With feature set: " + repr(feature_set)+ " Best accuracy: " + "%.2f" % best_accuracy + " %")
    print("Trivial feature/s: " + repr(feature_set2.difference(feature_set)))

In [14]:
def entry_func():
    print("Welcome to Xiao's feature search")
    file_name = input("Please enter file name: " )
    file_name = str(file_name)
    while True:
        search = input("Please select search algorithm:\n1 for Forward selection\n2 for Backward elimination\n3 for Custom algorithm\n4 to exit\n")
        search = int(search)
        if search == 4:
            print("Goodbye!")
            break
        print("Normalizing data...")
        raw_file_open = file_name
        data = process_file(raw_file_open)
        scaled_data = scale(data)
        print("Done")
        if search == 1:
            features = forward_selection(scaled_data)
        if search == 2:
            features = backward_elimination(scaled_data)
        if search == 3:
            custom_selection(scaled_data)
            

In [15]:
entry_func()

Welcome to Xiao's feature search
Please enter file name: CS170BIGtestdata__27.txt
Please select search algorithm:
1 for Forward selection
2 for Backward elimination
3 for Custom algorithm
4 to exit
1
Normalizing data...
Done
Froward selection
At level: 1
Best accuracy so far: 88.24 % With feature set: {31}
At level: 2
Best accuracy so far: 95.93 % With feature set: {3, 31}
At level: 3
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 4
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 5
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 6
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 7
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 8
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 9
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
A

(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 40
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 41
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 42
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 43
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 44
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 45
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 46
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 47
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 48
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 49
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 50
(Warning, 

(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 25
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 26
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 27
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 28
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 29
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 30
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 31
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 32
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 33
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 34
(Warning, Accuracy has decreased! Continuing search in case of local maxima)
At level: 35
(Warning, 